# Import modules and functions

In [52]:
import os
from owslib.wfs import WebFeatureService
import shapely.wkt
import geopandas as gpd
import json
from pathlib import Path
import urllib
import gzip
import pandas as pd

import warnings
# Ignore Futurewarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_columns', None) #-- to show all columns of datasets. To reset: pd.reset_option(“max_columns”)

In [2]:
GPKG_URL = "https://data.3dbag.nl/gpkg/v210908_fd2cee53/3dbag_v210908_fd2cee53_{TID}.gpkg.gz"
WFS_URL = "https://data.3dbag.nl/api/BAG3D_v2/wfs"
WFS_LAYER = "BAG3D_v2:bag_tiles_3k"

In [3]:
def get_tile_ids(wfs_url, wfs_layer, bbox):
    wfs11 = WebFeatureService(url=wfs_url, version='1.1.0')
    response = wfs11.getfeature(typename=wfs_layer, bbox=bbox, srsname='urn:x-ogc:def:crs:EPSG:28992', outputFormat='json')

    tiles = json.loads( response.read().decode('utf-8') )['features']
    tile_ids = [ tile['properties']['tile_id'] for tile in tiles ]

    return tile_ids

def download_3dbag(tile_ids, tilesdir):
    fnames = []
    for tid in tile_ids:
        url = GPKG_URL.format(TID=tid)
        # print(url)
        fname = tilesdir / (tid + '.gpkg')
        try:
            with urllib.request.urlopen(url) as response, open(fname, 'wb') as out_file:
                data = response.read()  # a `bytes` object
                out_file.write(gzip.decompress(data))
                fnames.append(fname)
        except urllib.error.HTTPError as err:
            print(err)

    return fnames

## Load study area extent

In [4]:
#Load study area extent - both Rdam and AMS
admin_area = gpd.read_file('../data/raw_data/admin_area/gadm41_NLD_2.json').to_crs(28992)

# Create a list containing the placenames as they are in the dataset.
# Den Haag is 's-Gravenhage' in the NL admin area dataset. Den Haag is the 'modern' abbreviation.
places_list = ['Rotterdam', 'Utrecht', '\'s-Gravenhage', 'Groningen', 'Heerhugowaard',
       'Amsterdam', 'Hendrik-Ido-Ambacht', 'Dordrecht']

study_areas = admin_area[admin_area['NAME_2'].isin(places_list)]

print(f"To confirm that this worked: We're left with the following cities {list(study_areas.NAME_2.unique())}")


def get_bounds(row):
    return row['geometry'].bounds

study_areas['bounds'] = study_areas.apply(get_bounds, axis=1)

To confirm that this worked: We're left with the following cities ['Groningen', 'Amsterdam', 'Heerhugowaard', 'Utrecht', "'s-Gravenhage", 'Dordrecht', 'Hendrik-Ido-Ambacht', 'Rotterdam']


/opt/anaconda3/envs/geo_env/lib/python3.10/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


## Get the tile id's

In [5]:
# Create an empty list
tile_ids_temp = []

# Iterate through the study_area to get all relevant tile_ids
for areas in study_areas.itertuples():
    tile_ids_layer = get_tile_ids(WFS_URL, WFS_LAYER, areas.bounds)
    print(f'(Study area {areas.NAME_2} was processed and {len(tile_ids_layer)} tiles were added to the tile_id list')
    tile_ids_temp.append(tile_ids_layer)

# The above loop created a nested list. This next piece is used to flatten the list so we can use it to query.
tile_ids = []
for sublist in tile_ids_temp:
    for item in sublist:
        tile_ids.append(item)
        
print('------------------')
print(f'Congratulations, a total of {len(tile_ids)} tiles was added to the list')

(Study area Groningen was processed and 116 tiles were added to the tile_id list
(Study area Amsterdam was processed and 247 tiles were added to the tile_id list
(Study area Heerhugowaard was processed and 41 tiles were added to the tile_id list
(Study area Utrecht was processed and 179 tiles were added to the tile_id list
(Study area 's-Gravenhage was processed and 188 tiles were added to the tile_id list
(Study area Dordrecht was processed and 86 tiles were added to the tile_id list
(Study area Hendrik-Ido-Ambacht was processed and 33 tiles were added to the tile_id list
(Study area Rotterdam was processed and 413 tiles were added to the tile_id list
------------------
Congratulations, a total of 1303 tiles was added to the list


In [6]:
# Download the 3dbag tiles as gpkg. 
filenames = download_3dbag(tile_ids, Path("../data/clean_data/3dbag").resolve())

In [13]:
# Find layer names
gpkg = '../data/clean_data/3dbag/28.gpkg'
layers = fiona.listlayers(gpkg)
for layer in layers:
    print(layer)

pand
ondergrond
lod12_2d
lod12_3d
lod13_2d
lod13_3d
lod22_2d
lod22_3d


In [93]:
pd.set_option('display.max_columns', None)

In [95]:
test_file = gpd.read_file('../data/clean_data/3dbag/tiles/2981.gpkg', layer='lod12_2d')
test_file.head()

,gid,fid,pand_deel_id,dd_id,h_dak_min,h_dak_50p,h_dak_70p,h_dak_max,ondergronds_type,tile_id,geometry
0,21099631,5614198,0,0,0.487889,6.869765,10.828572,12.847837,above ground,2981,"POLYGON ((80867.453 456066.917, 80867.900 4560..."
1,21099646,5614199,0,0,1.123393,15.737561,16.023325,17.762978,above ground,2981,"POLYGON ((80986.556 456120.419, 80985.418 4561..."
2,21100120,5616400,0,0,1.991461,5.265019,5.276763,5.362793,above ground,2981,"POLYGON ((80683.457 455920.311, 80687.773 4559..."
3,21100232,5617472,0,0,0.148521,13.977546,14.139849,16.385593,above ground,2981,"POLYGON ((80942.034 456212.153, 80943.004 4562..."
4,21100435,5618500,0,0,1.067675,10.575963,11.144601,12.088140,above ground,2981,"POLYGON ((80881.945 456203.405, 80881.920 4562..."


In [96]:
## Calculate building height and save only lod2.2 2D layer as separate file
# path to folder containing .gpkg files
folder_path = '../data/clean_data/3dbag/tiles'
export_path = '../data/clean_data/3dbag/heights'
# iterate through each .gpkg file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".gpkg"):
        file_path = os.path.join(folder_path, filename)
        # read the file into a GeoDataFrame
        gdf = gpd.read_file(file_path, driver='GPKG', layer='lod12_2d')
        # add a new column to the GeoDataFrame
        gdf['height_70p'] = (gdf['h_dak_70p'] - gdf['h_dak_min'])
        
        gdf = gdf.drop(columns=['fid'])
        
        save_path = os.path.join(export_path, filename)
        # save the updated GeoDataFrame to the same file, ignoring errors (empty tiles)
        try:
            gdf.to_file(save_path, driver='GPKG', layer='lod12_2d')
        except Exception as e:
            print(f'Error saving file {filename}: {e}')

Error saving file 143.gpkg: Cannot write empty DataFrame to file.


In [97]:
## Combine all gpkg files into one
folder_path = "../data/clean_data/3dbag/heights"
export_path = "../data/clean_data/3dbag/buildings_study_area.gpkg"

gdf_list = []

# iterate through each .gpkg file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".gpkg"):
        file_path = os.path.join(folder_path, filename)
        # read the file into a GeoDataFrame
        try:
            gdf = gpd.read_file(file_path, driver='GPKG', layer='lod12_2d')
            gdf_list.append(gdf)
        except Exception as e:
            print(f'Error appending file {filename}: {e}')

# combine all GeoDataFrames into one
combined_gdf = gpd.GeoDataFrame(pd.concat(gdf_list), crs="EPSG:28992")

# save the combined GeoDataFrame to a new file
combined_gdf.to_file(export_path, driver='GPKG', layer='lod12_2d')

In [79]:
export_path = "../data/clean_data/3dbag/buildings_study_area.gpkg"
# save the combined GeoDataFrame to a new file (added as layer to the file)
combined_gdf.to_file(export_path, driver='GPKG', layer='lod12_2d')

Split into city parts

In [98]:
data_dict = {}

# Take the study areas dataframe and iterate over it to clip the buildings into individual gpkg files.
for index, row in study_areas.iterrows():
    area_name = row['NAME_2']
    print(area_name)
    area_gdf = combined_gdf.clip(row.geometry)
    try:
        area_gdf.to_file(f'../data/clean_data/3dbag/{area_name}_buildings.gpkg', driver='GPKG')
    except:
        area_gdf.to_file(f'../data/clean_data/3dbag/{area_name[1:]}_buildings.gpkg', driver='GPKG')
    # data_dict[area_name] = area_gdf

Groningen
Amsterdam
Heerhugowaard
Utrecht
's-Gravenhage
Dordrecht
Hendrik-Ido-Ambacht
Rotterdam


In [102]:
testing = gpd.read_file('../data/clean_data/3dbag/Groningen_buildings.gpkg')
testing.head()

,gid,pand_deel_id,dd_id,h_dak_min,h_dak_50p,h_dak_70p,h_dak_max,ondergronds_type,tile_id,height_70p,geometry
0,22139603,0.0,0,4.585925,10.246358,10.253629,10.442611,above ground,618,5.667704,"POLYGON ((240055.040 569949.481, 240056.503 56..."
1,22140879,0.0,0,3.167362,6.226873,6.233937,6.295001,above ground,618,3.066575,"POLYGON ((240039.835 569959.015, 240037.914 56..."
2,22140348,0.0,0,4.675286,7.805014,7.901256,8.142466,above ground,618,3.225969,"POLYGON ((239849.569 570003.672, 239849.588 57..."
3,22138667,0.0,0,6.318471,9.079974,9.181605,9.634688,above ground,618,2.863134,"POLYGON ((240044.663 569997.784, 240044.725 56..."
4,22140328,0.0,0,7.165605,9.424167,9.737809,10.309152,above ground,618,2.572204,"POLYGON ((239934.509 570002.524, 239934.425 57..."


In [ ]:
testing.explore(column='height_70p')